> 50. Pow(x, n)

In [ ]:
class Solution:
    def myPow(self, x: float, n: int) -> float:
        # use a for loop over n and each time multiply x? --> brute force
        # use the fact that x^(2n) = x^n * x^n with recursion
        if not n:
            return 1
        
        if n < 0:
            return float(1 / self.myPow(x, -n))
        
        if n % 2 == 0:
            return self.myPow(x * x, n//2)
        
        else:
            return self.myPow(x * x, n//2) * x